In [1]:
import pandas as pd
import cv2
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
train=pd.read_csv("sign_mnist_train.csv")
test=pd.read_csv("sign_mnist_test.csv")

In [3]:
y_train=train['label']
y_test=test['label']
del train['label']
del test['label']

In [4]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

In [5]:
x_train = train.values
x_test = test.values

In [6]:
x_train = x_train / 255
x_test = x_test / 255

In [7]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

datagen.fit(x_train)

In [10]:
from keras.models import Sequential
from keras.layers import Activation,Dropout,Flatten,Dense,Conv2D,MaxPool2D,BatchNormalization

In [11]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))

In [12]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 75)        750       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 75)        300       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 50)        33800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 50)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 50)        200       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 50)          0         
__________

In [14]:
history=model.fit_generator(datagen.flow(x_train,y_train,batch_size=128),epochs = 8)

Epoch 1/8
215/215 [==============================] - 84s 392ms/step - loss: 1.0631 - acc: 0.6625
Epoch 2/8
215/215 [==============================] - 82s 380ms/step - loss: 0.2295 - acc: 0.9227
Epoch 3/8
215/215 [==============================] - 83s 387ms/step - loss: 0.1091 - acc: 0.9657
Epoch 4/8
215/215 [==============================] - 83s 385ms/step - loss: 0.0703 - acc: 0.9775
Epoch 5/8
215/215 [==============================] - 82s 380ms/step - loss: 0.0496 - acc: 0.9838
Epoch 6/8
215/215 [==============================] - 82s 382ms/step - loss: 0.0385 - acc: 0.9875
Epoch 7/8
215/215 [==============================] - 82s 382ms/step - loss: 0.0328 - acc: 0.9891
Epoch 8/8
215/215 [==============================] - 82s 383ms/step - loss: 0.0287 - acc: 0.9904


In [17]:
model.save('ml2.h5')

In [15]:
test_loss,test_accuracy=model.evaluate(x_test,y_test)

7172/7172 [==============================] - 8s 1ms/step


In [16]:
print("Test loss:",test_loss)
print("Test accuracy:",test_accuracy)

Test loss: 0.14096831349180294
Test accuracy: 0.9461795872838817
